In [1]:
import os
import sys
import glob
import re
import ast
import warnings

import csv
import json
import pickle

import math
import random
import numpy as np
import scipy as sp
import datetime as dt
import pandas as pd
import swifter
from scipy.stats import gaussian_kde
from scipy.integrate import quad
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

import portion as P
import itertools as it
import copy
from tqdm.notebook import tqdm
from collections import namedtuple
from pprint import pprint
from pytictoc import TicToc

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
%config InlineBackend.figure_format = 'retina'

# Set plot style
# plt.style.use('ggplot')

# Formulas

In [2]:
def estimate_conditional_parameters(data1, data2):
    data = {
        'first_distribution_samples': np.array(data1),  # 第一個分布的樣本數據
        'second_distribution_samples': np.array(data2)  # 第二個分布的樣本數據
    }
    
    # 定義對數似然函數
    def log_likelihood(params, data):
        p, success_prob_when_first_is_success, success_prob_when_first_is_failure = params
        first_samples = data['first_distribution_samples']
        second_samples = data['second_distribution_samples']
        
        log_likelihood_val = 0.0
        for first_sample, second_sample in zip(first_samples, second_samples):
            # 添加檢查以確保概率值在有效範圍內
            if not 0 <= success_prob_when_first_is_success <= 1:
                return float('inf')  # 返回無窮大值表示非法參數
            if not 0 <= success_prob_when_first_is_failure <= 1:
                return float('inf')  # 返回無窮大值表示非法參數
            # 計算對數似然函數值
            if first_sample == 1:
                log_likelihood_val += np.log(success_prob_when_first_is_success if second_sample == 1 else (1 - success_prob_when_first_is_success))
            else:
                log_likelihood_val += np.log(success_prob_when_first_is_failure if second_sample == 1 else (1 - success_prob_when_first_is_failure))
                
        return -log_likelihood_val  # 取負對數似然函數，因為我們要最大化對數似然函數

    # 使用最大似然估計估算參數
    initial_guess = [0.5, 0.5, 0.5]  # 初始猜測值
    result = minimize(log_likelihood, initial_guess, args=(data,), method='Nelder-Mead')

    # 輸出估計的參數值
    estimated_params = result.x
    p = np.mean(data1)
    alpha = estimated_params[1]
    beta = estimated_params[2]
    return p, alpha, beta

def estimate_joint_parameters(data1, data2):
    p = np.mean(data1)
    q = np.mean(data2)
    data = [(element1, element2) for element1, element2 in zip(data1, data2)]
    a = np.mean([1 if pair == (1, 1) else 0 for pair in data])
    b = np.mean([1 if pair == (1, 0) else 0 for pair in data])
    c = np.mean([1 if pair == (0, 1) else 0 for pair in data])
    d = np.mean([1 if pair == (0, 0) else 0 for pair in data])
    return p, q, a, b, c, d

In [3]:
def joint_to_conditional(a, b, c, d):
    """
    a := P(X = 1, Y = 1)
    b := P(X = 1, Y = 0)
    c := P(X = 0, Y = 1)
    d := P(X = 0, Y = 0)
    alpha := P(Y = 1 | X = 1)
    beta  := P(Y = 1 | X = 0)
    gamma := P(X = 1 | Y = 1)
    delta := P(X = 1 | Y = 0)
    """
    p = min(max(1e-9, a + b), 1 - 1e-9)
    q = min(max(1e-9, a + c), 1 - 1e-9)
    alpha = a / p
    beta = c / (1 - p)
    gamma = a / q
    delta = b / (1 - q)
    return alpha, beta, gamma, delta

def contional_to_joint(p, q, alpha, beta, gamma, delta):
    """
    a := P(X = 1, Y = 1)
    b := P(X = 1, Y = 0)
    c := P(X = 0, Y = 1)
    d := P(X = 0, Y = 0)
    alpha := P(Y = 1 | X = 1)
    beta  := P(Y = 1 | X = 0)
    gamma := P(X = 1 | Y = 1)
    delta := P(X = 1 | Y = 0)
    """
    a = p * alpha
    b = (1 - q) * delta
    c = (1 - p) * beta
    d = 1 - a - b - c
    return a, b, c, d

def calculate_rho_conditional(p, q, alpha):
    """
    p := P(X = 1); P(X = 0) = 1 - p
    q := P(Y = 1); P(Y = 0) = 1 - q
    alpha := P(Y = 1 | X = 1); P(Y = 0 | X = 1) = 1 - alpha
    rho: correlation coefficient
    """
    sigma = max(math.sqrt(p * q * (1 - p) * (1 - q)), 1e-9)  # sigma_x * sigma_y
    rho = (p * alpha - p * q) / sigma
    return rho

def calculate_rho_joint(p, q, a):
    """
    p := P(X = 1); P(X = 0) = 1 - p
    q := P(Y = 1); P(Y = 0) = 1 - q
    a := P(X = 1, Y = 1)
    rho: correlation coefficient
    """
    sigma = max(math.sqrt(p * q * (1 - p) * (1 - q)), 1e-9)  # sigma_x * sigma_y
    rho = (a - p * q) / sigma
    return rho

def calculate_conditional_probabilities(p, q, rho):
    """
    p := P(X = 1); P(X = 0) = 1 - p
    q := P(Y = 1); P(Y = 0) = 1 - q
    rho: correlation coefficient
    alpha := P(Y = 1 | X = 1)
    beta  := P(Y = 1 | X = 0)
    gamma := P(X = 1 | Y = 1)
    delta := P(X = 1 | Y = 0)
    """
    sigma = max(math.sqrt(p * q * (1 - p) * (1 - q)), 1e-9)  # sigma_x * sigma_y
    p = min(max(1e-9, p), 1 - 1e-9)
    q = min(max(1e-9, q), 1 - 1e-9)
    alpha = (p * q + rho * sigma) / p
    beta = (q * (1 - p) - rho * sigma) / (1 - p)
    gamma = (p * q + rho * sigma) / q
    delta = (p * (1 - q) - rho * sigma) / (1 - q)
    return alpha, beta, gamma, delta

def calculate_joint_probabilities(p, q, rho):
    """
    p := P(X = 1); P(X = 0) = 1 - p
    q := P(Y = 1); P(Y = 0) = 1 - q
    rho: correlation coefficient
    a := P(X = 1, Y = 1)
    b := P(X = 1, Y = 0)
    c := P(X = 0, Y = 1)
    d := P(X = 0, Y = 0)
    """
    sigma = max(math.sqrt(p * q * (1 - p) * (1 - q)), 1e-9)  # sigma_x * sigma_y
    a = p * q + rho * sigma
    b = p * (1 - q) - rho * sigma
    c = q * (1 - p) - rho * sigma
    d = (1 - p) * (1 - q) + rho * sigma
    return a, b, c, d

In [4]:
def generate_random_boolean(probability_true):
    return 1 if random.random() < probability_true else 0

def generate_data_single(p, size=10000):
    data = np.array([generate_random_boolean(p) for _ in range(size)])
    return data

def generate_data_conditional(p, q, rho, size=10000):
    alpha, beta, gamma, delta = calculate_conditional_probabilities(p, q, rho)
    
    # a, b, c, d = contional_to_joint(p, q, alpha, beta, gamma, delta)
    # print("p, q, rho:", p, q, rho)
    # print("alpha, beta, gamma, delta:", alpha, beta, gamma, delta)
    # print("a, b, c, d:", a, b, c, d)
    # print(sum([a, b, c, d]))
    
    data1 = np.array([generate_random_boolean(p) for _ in range(size)])
    data2 = np.array([generate_random_boolean(alpha) if element == 1 else generate_random_boolean(beta) for element in data1])
    return data1, data2

def generate_data_joint(p, q, rho, size=10000):
    a, b, c, d = calculate_joint_probabilities(p, q, rho)
    events = [1, 2, 3, 4]
    probabilities = [a, b, c, d]
    
    # alpha, beta, gamma, delta = joint_to_conditional(a, b, c, d)
    # print("p, q, rho:", p, q, rho)
    # print("a, b, c, d:", a, b, c, d)
    # print("alpha, beta, gamma, delta:", alpha, beta, gamma, delta)
    # print(sum([a, b, c, d]))
    
    premiere_data = np.random.choice(events, size=size, p=probabilities)
    data = [(1 if element in [1, 2] else 0, 1 if element in [1, 3] else 0) for element in premiere_data]
    data1 = np.array([pair[0] for pair in data])
    data2 = np.array([pair[1] for pair in data])
    return data1, data2

In [5]:
def rho_restriction(p, q):
    sigma = max(math.sqrt(p * q * (1 - p) * (1 - q)), 1e-9)  # sigma_x * sigma_y
    R1 = P.closed(-1, 1)  # -1 <= rho <= 1
    R2 = P.closed(-(p * q) / sigma, (1 - p * q) / sigma)  # 0 <= P(X=1, Y=1) <= 1
    R3 = P.closed((p * (1 - q) - 1) / sigma, p * (1 - q) / sigma)  # 0 <= P(X=1, Y=0) <= 1
    R4 = P.closed((q * (1 - p) - 1) / sigma, q * (1 - p) / sigma)  # 0 <= P(X=0, Y=1) <= 1
    R5 = P.closed(-((1 - p) * (1 - q)) / sigma, (1 - (1 - p) * (1 - q)) / sigma)  # 0 <= P(X=0, Y=0) <= 1
    R = R1 & R2 & R3 & R4 & R5
    return R

# rho's restrictions

In [6]:
p = 0.99
q = 0.3
n = 20
rho_bound = rho_restriction(p, q)
print("rho restriction:", rho_bound)
lower_bd = rho_bound.lower
upper_bd = rho_bound.upper
step = (upper_bd - lower_bd) / n

table = pd.DataFrame(columns="p, q, rho, alpha, beta, gamma, delta, a, b, c, d, sum".split(", "))
rho = lower_bd
for i in range(n + 1):
    alpha, beta, gamma, delta = calculate_conditional_probabilities(p, q, rho)
    a, b, c, d = calculate_joint_probabilities(p, q, rho)
    probability = [p, q, rho, alpha, beta, gamma, delta, a, b, c, d, sum([a, b, c, d])]
    for i, item in enumerate(probability):
        probability[i] = round(item, 3)
    table.loc[len(table)] = probability    
    rho += step

display(table)

rho restriction: [-0.15352206215727948,0.06579516949597693]


,p,q,rho,alpha,beta,gamma,delta,a,b,c,d,sum
0,0.99,0.3,-0.154,0.293,1.00,0.967,1.000,0.290,0.700,0.010,0.000,1.0
1,0.99,0.3,-0.143,0.293,0.95,0.968,0.999,0.290,0.699,0.010,0.001,1.0
2,0.99,0.3,-0.132,0.294,0.90,0.970,0.999,0.291,0.699,0.009,0.001,1.0
3,0.99,0.3,-0.121,0.294,0.85,0.972,0.998,0.291,0.698,0.009,0.002,1.0
4,0.99,0.3,-0.110,0.295,0.80,0.973,0.997,0.292,0.698,0.008,0.002,1.0
5,0.99,0.3,-0.099,0.295,0.75,0.975,0.996,0.292,0.697,0.008,0.003,1.0
6,0.99,0.3,-0.088,0.296,0.70,0.977,0.996,0.293,0.697,0.007,0.003,1.0
7,0.99,0.3,-0.077,0.296,0.65,0.978,0.995,0.293,0.696,0.007,0.004,1.0
8,0.99,0.3,-0.066,0.297,0.60,0.980,0.994,0.294,0.696,0.006,0.004,1.0
9,0.99,0.3,-0.055,0.297,0.55,0.982,0.994,0.294,0.696,0.006,0.005,1.0


# Cross validate rhos

In [24]:
# 測試函數
# data1 = np.random.randint(2, size=10000)
# data2 = np.random.randint(2, size=10000)

# 隨機生成序列1
data1 = np.random.randint(2, size=10000)  # 生成長度為 10 的 {0, 1} 序列
# 創建具有相關性的序列2
correlation = 0.3  # 相關性的強度
# 對序列1進行微調以生成序列2
data2 = np.array([1 if np.random.rand() < correlation * element else 0 for element in data1])

print("Random sequence 1:", data1)
print("Random sequence 2:", data2)

rho = np.corrcoef(data1[:], data2[:])[0, 1]
print("rho:", rho)

Random sequence 1: [0 0 0 ... 1 1 1]
Random sequence 2: [0 0 0 ... 1 0 0]
rho: 0.4166259691180373


In [25]:
p, alpha, beta = estimate_conditional_parameters(data1[:], data2[:])
q, gamma, delta = estimate_conditional_parameters(data2[:], data1[:])
rho1 = calculate_rho_conditional(p, q, alpha)
rho2 = calculate_rho_conditional(q, p, gamma)

print("Use simple average to estimate p, q")
print("Use maximum likelihood to estimate alpha, beta, gamma, delta")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"alpha := P(Y=1 | X=1) {alpha}")
print(f"beta  := P(Y=1 | X=0) {beta}")
print(f"gamma := P(X=1 | Y=1) {gamma}")
print(f"delta := P(X=1 | Y=0) {delta}")
print("rho:", rho, rho1, rho2)
print("----------------------------------------------------------")

Use simple average to estimate p, q
Use maximum likelihood to estimate alpha, beta, gamma, delta
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.4986
q := P(Y=1): 0.1472
alpha := P(Y=1 | X=1) 0.29520782460337663
beta  := P(Y=1 | X=0) 6.162267454387372e-09
gamma := P(X=1 | Y=1) 0.9999999987129929
delta := P(X=1 | Y=0) 0.41206179415981725
rho: 0.4166259691180373 0.41657302781166294 0.4166259680486319
----------------------------------------------------------


In [26]:
p, q, a, b, c, d = estimate_joint_parameters(data1[:], data2[:])
rho1 = calculate_rho_joint(p, q, a)

print("Use simple average to estimate p, q, a, b, c, d")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"a := P(X=1, Y=1) {a}")
print(f"b := P(X=1, Y=0) {b}")
print(f"c := P(X=0, Y=1) {c}")
print(f"d := P(X=0, Y=0) {d}")
print("rho:", rho, rho1)
print("----------------------------------------------------------")

Use simple average to estimate p, q, a, b, c, d
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.4986
q := P(Y=1): 0.1472
a := P(X=1, Y=1) 0.1472
b := P(X=1, Y=0) 0.3514
c := P(X=0, Y=1) 0.0
d := P(X=0, Y=0) 0.5014
rho: 0.4166259691180373 0.41662596911803856
----------------------------------------------------------


# Generate data: conditional

In [60]:
p, q = 0.9, 0.3
rho_bound = rho_restriction(p, q)
print("rho restriction:", rho_bound)
rho = 0.1

data1, data2 = generate_data_conditional(p, q, rho)

print("----------------------------------------------------------")
print("Random sequence 1:", data1)
print("Random sequence 2:", data2)
print("p, q, rho:", p, q, rho)

rho = np.corrcoef(data1[:], data2[:])[0, 1]
print("----------------------------------------------------------")
print("rho:", rho)

rho restriction: [-0.5091750772173154,0.21821789023599233]
----------------------------------------------------------
Random sequence 1: [1 1 1 ... 1 1 1]
Random sequence 2: [1 0 0 ... 0 1 1]
p, q, rho: 0.9 0.3 0.1
----------------------------------------------------------
rho: 0.10932930790053134


In [61]:
p, alpha, beta = estimate_conditional_parameters(data1[:], data2[:])
q, gamma, delta = estimate_conditional_parameters(data2[:], data1[:])
rho1 = calculate_rho_conditional(p, q, alpha)
rho2 = calculate_rho_conditional(q, p, gamma)

print("Use simple average to estimate p, q")
print("Use maximum likelihood to estimate alpha, beta, gamma, delta")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"alpha := P(Y=1 | X=1) {alpha}")
print(f"beta  := P(Y=1 | X=0) {beta}")
print(f"gamma := P(X=1 | Y=1) {gamma}")
print(f"delta := P(X=1 | Y=0) {delta}")
print("rho:", rho, rho1, rho2)
print("----------------------------------------------------------")

Use simple average to estimate p, q
Use maximum likelihood to estimate alpha, beta, gamma, delta
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.8966
q := P(Y=1): 0.306
alpha := P(Y=1 | X=1) 0.32310764321429286
beta  := P(Y=1 | X=0) 0.15763420110281914
gamma := P(X=1 | Y=1) 0.9467319665458788
delta := P(X=1 | Y=0) 0.8744781665800074
rho: 0.10932930790053134 0.1093172841706095 0.10932917792775795
----------------------------------------------------------


In [62]:
p, q, a, b, c, d = estimate_joint_parameters(data1[:], data2[:])
rho1 = calculate_rho_joint(p, q, a)

print("Use simple average to estimate p, q, a, b, c, d")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"a := P(X=1, Y=1) {a}")
print(f"b := P(X=1, Y=0) {b}")
print(f"c := P(X=0, Y=1) {c}")
print(f"d := P(X=0, Y=0) {d}")
print("rho:", rho, rho1)
print("----------------------------------------------------------")

Use simple average to estimate p, q, a, b, c, d
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.8966
q := P(Y=1): 0.306
a := P(X=1, Y=1) 0.2897
b := P(X=1, Y=0) 0.6069
c := P(X=0, Y=1) 0.0163
d := P(X=0, Y=0) 0.0871
rho: 0.10932930790053134 0.10932930790053155
----------------------------------------------------------


# Generate data: joint

In [63]:
p, q = 0.9, 0.3
rho_bound = rho_restriction(p, q)
print("rho restriction:", rho_bound)
rho = 0.1

data1, data2 = generate_data_joint(p, q, rho)

print("----------------------------------------------------------")
print("Random sequence 1:", data1)
print("Random sequence 2:", data2)
print("p, q, rho:", p, q, rho)

rho = np.corrcoef(data1[:], data2[:])[0, 1]
print("----------------------------------------------------------")
print("rho:", rho)

rho restriction: [-0.5091750772173154,0.21821789023599233]
----------------------------------------------------------
Random sequence 1: [1 1 1 ... 1 1 1]
Random sequence 2: [0 1 0 ... 1 0 0]
p, q, rho: 0.9 0.3 0.1
----------------------------------------------------------
rho: 0.0932618097602564


In [64]:
p, alpha, beta = estimate_conditional_parameters(data1[:], data2[:])
q, gamma, delta = estimate_conditional_parameters(data2[:], data1[:])
rho1 = calculate_rho_conditional(p, q, alpha)
rho2 = calculate_rho_conditional(q, p, gamma)

print("Use simple average to estimate p, q")
print("Use maximum likelihood to estimate alpha, beta, gamma, delta")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"alpha := P(Y=1 | X=1) {alpha}")
print(f"beta  := P(Y=1 | X=0) {beta}")
print(f"gamma := P(X=1 | Y=1) {gamma}")
print(f"delta := P(X=1 | Y=0) {delta}")
print("rho:", rho, rho1, rho2)
print("----------------------------------------------------------")

Use simple average to estimate p, q
Use maximum likelihood to estimate alpha, beta, gamma, delta
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.9012
q := P(Y=1): 0.3013
alpha := P(Y=1 | X=1) 0.3154698730544553
beta  := P(Y=1 | X=0) 0.1720505755708001
gamma := P(X=1 | Y=1) 0.9435862468466416
delta := P(X=1 | Y=0) 0.8829359277566222
rho: 0.0932618097602564 0.09327239774197793 0.09328033420493372
----------------------------------------------------------


In [65]:
p, q, a, b, c, d = estimate_joint_parameters(data1[:], data2[:])
rho1 = calculate_rho_joint(p, q, a)

print("Use simple average to estimate p, q, a, b, c, d")
print("----------------------------------------------------------")
print("Estimated parameters:")
print(f"p := P(X=1): {p}")
print(f"q := P(Y=1): {q}")
print(f"a := P(X=1, Y=1) {a}")
print(f"b := P(X=1, Y=0) {b}")
print(f"c := P(X=0, Y=1) {c}")
print(f"d := P(X=0, Y=0) {d}")
print("rho:", rho, rho1)
print("----------------------------------------------------------")

Use simple average to estimate p, q, a, b, c, d
----------------------------------------------------------
Estimated parameters:
p := P(X=1): 0.9012
q := P(Y=1): 0.3013
a := P(X=1, Y=1) 0.2843
b := P(X=1, Y=0) 0.6169
c := P(X=0, Y=1) 0.017
d := P(X=0, Y=0) 0.0818
rho: 0.0932618097602564 0.09326180976025573
----------------------------------------------------------
